In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import fitz
import pytesseract
from PIL import Image

In [3]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [4]:
os.environ['PATH'] += r';C:\Program Files\Tesseract-OCR'

In [205]:
# In case of .pdf file
pages = []
file = '6_part.pdf'
doc = fitz.open(file)
for n in range(doc.page_count):
    page = doc.load_page(n)
    pix = page.get_pixmap()
    image = np.frombuffer(pix.samples,
                          dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    image = np.ascontiguousarray(image[..., [2, 1, 0]])
    pages.append(image)

In [206]:
# Remove color info
image = pages[0]
# Отображение изображения с выделенными ячейками
# cv2.imshow('Image with Cells', image)
# cv2.imwrite('image.jpg', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Для выделения таблицы из корпуса текста попробуем использовать cv2 с такими настройками

In [207]:
# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Отображение изображения с выделенными ячейками
# cv2.imshow('Image with Cells', gray_image)
# cv2.imwrite('gray_image.jpg', gray_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Применение порогового преобразования для бинаризации изображения
ret, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Применение морфологического преобразования для удаления шума и замыкания линий
kernel = np.ones((5, 5), np.uint8)
morph_image = cv2.morphologyEx(threshold_image, cv2.MORPH_CLOSE, kernel)

# Нахождение контуров на изображении
contours, hierarchy = cv2.findContours(morph_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Создание списка координат ограничивающих рамок ячеек
cell_boxes = []

# Фильтрация контуров и определение ограничивающих рамок ячеек

height, width, _ = image.shape

parts_image = []

for contour in contours:
    # Определение площади контура
    area = cv2.contourArea(contour)

    x, y, w, h = cv2.boundingRect(contour)

    # cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Фильтрация контуров по площади
    if area > 3000:
        # Определение ограничивающей рамки контура
        x, y, w, h = cv2.boundingRect(contour)

        # Добавление ограничивающей рамки в список
        cell_boxes.append((x, y, x + w, y + h))

        cropp_image = image[y:y+h, x:x+w]

        parts_image.append(cropp_image)

# Вывод координат ограничивающих рамок ячеек
for i, box in enumerate(cell_boxes):
    x1, y1, x2, y2 = box
    # cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # cv2.imshow('Image with Cells', parts_image[i])
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

# # Отображение изображения с выделенными ячейками
# cv2.imshow('Image with Cells', image)
# cv2.imwrite('image_boxes.jpg', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [208]:
# Загрузка изображения
image = parts_image[0]

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Распознавание текста с помощью Tesseract
text = pytesseract.image_to_string(gray_image, lang='rus')

# Получение ограничивающих рамок символов
boxes = pytesseract.image_to_boxes(gray_image, lang='rus')

# # Вывод распознанного текста
# print("Распознанный текст:")
# print(text)

# Разделение строки на список строк
box_lines = boxes.strip().split('\n')

# Преобразование каждой строки в список
boxes_list = [line.split() for line in box_lines]

# print(boxes_list)

# Так как cv2 и tesseract используют разные системы координат
height, width = gray_image.shape

# Список границ таблицы
borders = []

epsilon = 5

# Вывод ограничивающих рамок символов
for box in boxes_list:
    if box[0] == '~' and ((abs(int(box[1]) - int(box[3])) < epsilon) or (abs(int(box[2]) - int(box[4])) < epsilon)):
        x, y, w, h = int(box[1]), int(box[2]), int(box[3]), int(box[4])
        borders.append([x, y, w, h])
        # cv2.rectangle(image, (x, height - y), (w, height - h), (255, 0, 255), 1)

# Отображение изображения с ограничивающими рамками символов
# cv2.imshow('Image with Character Boxes', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Попробуем найти координаты всех узлов таблицы, по этим узлам восстановим прямоугольники ячеек

In [209]:
import itertools

# Находим все вертикальные и горизонтальные линии каркаса таблицы
vertical_lines = []
horizontal_lines = []

# Параметры допуска при котором считаем что линии ровные и пересекаются
epsilon = 10

for x1, y1, x2, y2 in borders:
    if abs(x1 - x2) < epsilon:  # Вертикальная линия
        vertical_lines.append((int((x1+x2)/2), y1, int((x1+x2)/2), y2))
    elif abs(y1 - y2) < epsilon:  # Горизонтальная линия
        horizontal_lines.append((x1, int((y1+y2)/2), x2, int((y1+y2)/2)))

# Находим координаты узлов таблицы
table_nodes = []
# for v_line, h_line in itertools.product(vertical_lines, horizontal_lines):
#     x, y = v_line[0], h_line[1]
#     print(x, y)
#     cv2.rectangle(image, (x, height - y), (x + 3, height - y + 3), (0, 0, 255), 1)
#     table_nodes.append((x, y))

for v_line, h_line in itertools.product(vertical_lines, horizontal_lines):
    v_x1, v_y1, v_x2, v_y2 = v_line
    h_x1, h_y1, h_x2, h_y2 = h_line

    # Проверяем, пересекаются ли прямые или заканчивается ли одна из них в окрестности
    if ((h_x1 - epsilon <= v_x1 <= h_x2 + epsilon or h_x1 - epsilon <= v_x2 <= h_x2 + epsilon) and
        (v_y1 - epsilon <= h_y1 <= v_y2 + epsilon or v_y1 - epsilon <= h_y2 <= v_y2 + epsilon)) or \
        (abs(h_x2 - v_x1) <= epsilon and v_y1 <= h_y1 <= v_y2) or \
        (abs(h_x2 - v_x2) <= epsilon and v_y1 <= h_y1 <= v_y2) or \
        (abs(h_x1 - v_x1) <= epsilon and v_y1 <= h_y1 <= v_y2) or \
        (abs(h_x1 - v_x2) <= epsilon and v_y1 <= h_y1 <= v_y2):
        # Добавляем координаты узла в список
        table_nodes.append((v_x1, h_y1))

for v_line, h_line in itertools.product(vertical_lines, horizontal_lines):
    v_x1, v_y1, v_x2, v_y2 = v_line
    h_x1, h_y1, h_x2, h_y2 = h_line

    if v_x1 <= epsilon or v_y1 <= epsilon or abs(v_x1 - width) <= epsilon or abs(v_y1 - height) <= epsilon:
        table_nodes.append((v_x1, v_y1))

    if v_x2 <= epsilon or v_y2 <= epsilon or abs(v_x2 - width) <= epsilon or abs(v_y2 - height) <= epsilon:
        table_nodes.append((v_x2, v_y2))

    if h_x1 <= epsilon or h_y1 <= epsilon or abs(h_x1 - width) <= epsilon or abs(h_y1 - height) <= epsilon:
        table_nodes.append((h_x1, h_y1))

    if h_x2 <= epsilon or h_y2 <= epsilon or abs(h_x2 - width) <= epsilon or abs(h_y2 - height) <= epsilon:
        table_nodes.append((h_x2, h_y2))

# Создаем копию списка table_nodes для модификации
modified_table_nodes = table_nodes.copy()

# Проходим по каждой точке в списке table_nodes
for i, node in enumerate(table_nodes):
    x, y = node
    count = 1
    avg_x, avg_y = x, y

    # Ищем точки в окрестности epsilon
    for j, other_node in enumerate(table_nodes):
        if i != j:  # Игнорируем текущую точку
            other_x, other_y = other_node

            # Проверяем, находится ли другая точка в окрестности epsilon
            if abs(other_x - x) <= epsilon and abs(other_y - y) <= epsilon:
                avg_x += other_x
                avg_y += other_y
                count += 1

    # Если найдены точки в окрестности, заменяем текущую точку на среднее значение
    if count > 1:
        avg_x /= count
        avg_y /= count
        modified_table_nodes[i] = (int(avg_x), int(avg_y))

# Заменяем исходный список table_nodes на модифицированный
table_nodes = modified_table_nodes

# Выводим координаты узлов на изображении
for node in table_nodes:
    x, y = node
    print(x, y)
    cv2.rectangle(image, (x, height - y), (x + 3, height - y + 3), (0, 0, 255), 1)

# Отображение изображения
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

61 332
61 212
61 212
61 174
61 123
61 73
61 35
184 332
184 312
184 234
184 212
184 212
184 174
184 123
184 73
184 35
347 332
347 312
347 234
347 212
347 212
347 174
347 123
347 73
347 35
407 332
407 300
407 281
407 234
407 234
407 212
407 212
407 174
407 123
407 73
407 35
238 312
238 234
238 212
238 212
238 174
238 123
238 73
238 35
294 312
294 234
294 212
294 212
294 174
294 123
294 73
294 35
560 300
560 281
560 234
560 234
560 212
560 212
560 174
560 123
560 73
560 35
459 281
459 234
459 234
459 212
459 212
459 174
459 123
459 73
459 35
514 281
514 234
514 234
514 212
514 212
514 174
514 123
514 73
514 35
2 332
605 332
606 300
605 234
605 234
2 212
605 212
2 212
605 212
3 174
605 174
2 123
605 123
3 73
605 73
3 35
605 35
2 332
605 332
606 300
605 234
605 234
2 212
605 212
2 212
605 212
3 174
605 174
2 123
605 123
3 73
605 73
3 35
605 35
61 0
2 332
605 332
61 0
61 0
606 300
61 0
61 0
605 234
61 0
605 234
61 0
2 212
605 212
61 0
2 212
605 212
61 0
3 174
605 174
61 0
2 123
605 123
61 0


In [211]:
# Замена координат, отличающихся менее чем на epsilon
rounded_nodes = np.round(np.array(table_nodes) / epsilon) * epsilon

# Подсчет уникальных значений координат x и y
unique_x = np.unique(rounded_nodes[:, 0])
unique_y = np.unique(rounded_nodes[:, 1])

# Определение числа строк и столбцов таблицы
num_rows = len(unique_y) - 1
num_columns = len(unique_x) - 1

print(num_rows, num_columns)

# Отсортируем узлы по оси x
nodes_sorted_x = sorted(table_nodes, key=lambda x: (x[0], x[1]))

# Создадим список прямоугольников
rectangles = []

# Пройдем по узлам и создадим прямоугольники
for i in range(len(nodes_sorted_x) - 1):
    current_node = nodes_sorted_x[i]
    if nodes_sorted_x[i+1][0] == current_node[0]:
        next_y_node = nodes_sorted_x[i+1]
    else:
        continue
    # Найдем узлы следующие по оси y для текущих узлов
    next_x_nodes = [node for node in nodes_sorted_x if node[1] == current_node[1] and node[0] > current_node[0]]
    if next_x_nodes:
        next_x_node = min(next_x_nodes, key=lambda x: x[0])
        opposite_node = (next_x_node[0], next_y_node[1])
        # Построим прямоугольник по найденным узлам
        rectangle = ((current_node[0], current_node[1]), (opposite_node[0], opposite_node[1]))
        rectangles.append(rectangle)
        # cv2.rectangle(image, (current_node[0] + 2, height - current_node[1] + 2), (opposite_node[0] - 2, height - opposite_node[1] - 2), (255, 255, 0), 1)
    else:
        continue
# Выведем список прямоугольников
print(rectangles)

# # Отображение изображения
# cv2.imshow('Image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

10 11
[((2, 123), (61, 123)), ((2, 123), (61, 123)), ((2, 123), (61, 123)), ((2, 123), (61, 123)), ((2, 123), (61, 123)), ((2, 123), (61, 123)), ((2, 123), (61, 123)), ((2, 123), (61, 123)), ((2, 123), (61, 123)), ((2, 123), (61, 123)), ((2, 123), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((3,

In [212]:
rectangles = sorted(rectangles, key=lambda x: (- x[0][1], x[0][0]))
print(rectangles)

[((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((2, 332), (61, 332)), ((184, 312), (238, 332)), ((407, 300), (560, 332)), ((407, 281), (459, 300)), ((184, 234), (238, 312)), ((238, 234), (294, 312)), ((294, 234), (347, 312)), ((347, 234), (407, 312)), ((407, 234), (459, 234)), ((407, 234), (459, 281)), ((459, 234), (514, 234)), ((459, 234), (514, 281)), ((514, 234), (560, 234)), ((514, 234), (560, 281)), ((560, 234), (605, 234)), ((560, 234), (605, 281)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), (61, 212)), ((2, 212), 

In [215]:
# Создание пустого DataFrame для хранения данных таблицы
table_data = [['' for _ in range(num_columns)] for _ in range(num_rows)]

# Обход прямоугольников ячеек таблицы
for i, rectangle in enumerate(rectangles):
    # Извлечение координат углов прямоугольника
    x1, y1 = rectangle[0]
    x2, y2 = rectangle[1]

    # Обрезание изображения по границам ячейки
    cell_image = image[max(0, (height - y2) - 5):min(height, (height - y1) + 5),
                 max(0, x1 - 5):min(width, x2 + 5)]

    # Russian
    custom_config = r'--oem 1 --psm 1 -l rus'

    if cell_image.shape[0] > 10 and cell_image.shape[1] > 10:
        gray_image = cv2.cvtColor(cell_image, cv2.COLOR_BGR2GRAY)
        _, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY)

        # Отображение изображения
        cv2.imshow('Image', gray_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Распознавание текста с помощью Tesseract
        text = pytesseract.image_to_string(gray_image, config=custom_config).replace('\n', ' ')

        print(text)

        # Определение положения ячейки в таблице
        row = i // num_columns
        column = i % num_columns

        # Добавление данных ячейки
        if row < num_rows and column < num_columns:
            table_data[row][column] = text

# Вывод полученной таблицы
df = pd.DataFrame(table_data)
# Сохранение DataFrame в формате CSV
df.to_csv('table.csv', index=False, header=False)

"Механичес  
Ударная вязкость КСО, Д5 темы  
приз 
Прелел текучести, 1 Нун ‘бтеумы)  
[Временное `сопротив- ^ Нуые" клоуны)  
Относи- уалине-  ние &, 9  






"Толщина полки,  



бо сие  




От 40 20 включ. Св.20› 40 ›  
23504) 22503)  
360637) 360437)  
26 25  






От 4до 20 включ. Св.0, 5 › >50 >  
24505) 23509) 23504)  
370438) 37088) 370688)  
5 р р  
заа АИ 333  





От 4ло 10 включ. Св 020 > > 20.40 +  
25506) 24505) 23504)  
380439) 37088) 37038)  
5 5 м  
ъъь  ь5е 
29) 298) 296)  



‘275  
От 4до 10 включ. Св. 10>20 ›  
27508) 27508)  
390440) 38009)  
24 23  





4 о 10 включ. Ю» 20 
28509) 27528) 
400441) 390(40) 
4 3 
4-2 4=а 
29) 2963) 


